In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as spio
from scipy.stats import norm 
from scipy.stats import multivariate_normal
import time
import sys

flt_min = sys.float_info.min

%matplotlib inline

## EXTRA THINGS TO DO (Optional)
1. Most of the code you have so far (both provided by us and probably coded by you) is filled with computationally inefficient for-loops. Vectorize the code such that these for loops are one/two-liners with matrix multiplications. This will greatly speed up your code. 
2. Use the mixtures of Gaussians model to classify the skin/non-skin data. You need to use the same training data and inference model you used in part A along with all the 
    functions you've completed in part C to perform MoG training on multi-dimensional data. 
3. Use the Gaussian model (with diagonal covariance) to classify
face/non-face data (in FaceData.mat)
4. Use the mixtures of Gaussians model (with diagonal covariance) to
classify the face/non-face data
5. Use the t-distribution instead of the normal distribution


In [ ]:
#loading the image
im = plt.imread('bob_small.jpeg');
#loading the segmentation mask
gt = spio.loadmat('bob_GroundTruth_small.mat', squeeze_me=True)['gt']

In [ ]:
# load the file
trainingData = spio.loadmat('RGBSkinNonSkin.mat', squeeze_me=True)

#extract the non-skin matrix
RGBNonSkin = np.float32(trainingData['RGBNonSkin'])
#extract the skin matrix
RGBSkin = np.float32(trainingData['RGBSkin'])

In [71]:
def getMixGaussLogLike(data, mixGaussEst): 
    """
    Calculate the log likelihood for the whole dataset under a mixture of Gaussians model.
    
    Keyword arguments:
    data -- d by n matrix containing data points.
    mixGaussEst -- dict containing the mixture of gaussians parameters.

    Returns: 
    logLike -- scalar containing the log likelihood.
    
    """
    
    data = np.atleast_2d(data)                                                                         
    # find total number of data items                                                                  
    nDims, nData = data.shape                                                                          
    
    # initialize log likelihoods                                                                       
    logLike = 0;                                                                                       
                                                                                                       
    # run through each data item                                                                       
    for cData in range(nData):                                                                         
        thisData = data[:, cData]                                                                      
        # TO DO - calculate likelihood of this data point under mixture of                         
        # Gaussians model. Replace this                                                                
        like = 0
        for k in range(mixGaussEst['k']):
            weight = mixGaussEst['weight'][k]
            mu = mixGaussEst['mean'][:, k]
            cov = mixGaussEst['cov'][:,:,k]
            prob = (1 / ((2*np.pi)**(nDims/2) * np.sqrt(np.linalg.det(cov))) * 
                    np.exp(-0.5 * (thisData - mu).T @ np.linalg.inv(cov) @ (thisData - mu)))
            like += weight * prob
        
        # add to total log like                                                                        
        logLike = logLike + np.log(like)                                                               
                                                                                                       
    return  logLike.item()                                                                       
                                                                                                       

In [ ]:
def fitMixGauss(data, k):
    """
    Estimate a k MoG model that would fit the data. Incremently plots the outcome.
               
    
    Keyword arguments:
    data -- d by n matrix containing data points.
    k -- scalar representing the number of gaussians to use in the MoG model.
    
    Returns: 
    mixGaussEst -- dict containing the estimated MoG parameters.
    
    """

    #     MAIN E-M ROUTINE  
    #     In the E-M algorithm, we calculate a complete posterior distribution over                                  
    #     the (nData) hidden variables in the E-Step.  
    #     In the M-Step, we update the parameters of the Gaussians (mean, cov, w).   

    nDims, nData = data.shape

    postHidden = np.zeros(shape=(k, nData))

    # we will initialize the values to random values
    mixGaussEst = dict()
    mixGaussEst['d'] = nDims
    mixGaussEst['k'] = k
    mixGaussEst['weight'] = (1 / k) * np.ones(shape=(k))
    mixGaussEst['mean'] = 2 * np.random.randn(nDims, k)
    mixGaussEst['cov'] = np.random.randn(nDims, nDims, k)
    for cGauss in range(k):
        mixGaussEst['cov'][:, :, cGauss] = 2.5 + 1.5 * np.random.uniform() * np.eye(nDims)

    # calculate current likelihood
    # TO DO - fill in this routine
    logLike = getMixGaussLogLike(data, mixGaussEst)
    print('Log Likelihood Iter 0 : {:4.3f}\n'.format(logLike))

    nIter = 30;

    logLikeVec = np.zeros(shape=(2 * nIter))
    boundVec = np.zeros(shape=(2 * nIter))

    fig, ax = plt.subplots(1, 1)

    for cIter in range(nIter):

        # ===================== =====================
        # Expectation step
        # ===================== =====================

        for cData in range(nData):
            # TO DO (g) : fill in column of 'hidden' - calculate posterior probability that
            # this data point came from each of the Gaussians
            # replace this:
            thisData = data[:, cData]
            l = np.zeros((k, nData))
            for i in range(k):
                weight = mixGaussEst['weight'][i]
                mu = mixGaussEst['mean'][:, i]
                cov = mixGaussEst['cov'][:, :, i]
                prob = (1 / ((2 * np.pi) ** (nData / 2) * np.sqrt(np.linalg.det(cov))) *
                        np.exp(-0.5 * (thisData - mu).T @ np.linalg.inv(cov) @ (thisData - mu)))
                l[i] = weight * prob
            postHidden[:, cData] = l[:, cData] / np.sum(l[:, cData])

        # ===================== =====================
        # Maximization Step
        # ===================== =====================
        # for each constituent Gaussian
        for cGauss in range(k):
            # TO DO (h):  Update weighting parameters mixGauss.weight based on the total
            # posterior probability associated with each Gaussian. Replace this:
            mixGaussEst['weight'][cGauss] = np.sum(postHidden[cGauss, :]) / np.sum(postHidden)

            # TO DO (i):  Update mean parameters mixGauss.mean by weighted average
            # where weights are given by posterior probability associated with
            # Gaussian.  Replace this:
            tmp = 0
            for i in range(nData):
                tmp += postHidden[cGauss, i] * data[:, i]
            mixGaussEst['mean'][:, cGauss] = tmp / np.sum(postHidden[cGauss, :])

            # TO DO (j):  Update covarance parameter based on weighted average of
            # square distance from update mean, where weights are given by
            # posterior probability associated with Gaussian
            tmp = 0
            for i in range(nData):
                tmp += (postHidden[cGauss, i] * (data[:, i] - mixGaussEst['mean'][:, cGauss]).reshape(-1, 1) @ (
                            data[:, i] - mixGaussEst['mean'][:, cGauss]).reshape(1, -1))
            mixGaussEst['cov'][:, :, cGauss] = tmp / np.sum(postHidden[cGauss, :])

            # draw the new solution

        drawEMData2d(data, mixGaussEst)
        time.sleep(0.7)
        fig.canvas.draw()

        # calculate the log likelihood
        logLike = getMixGaussLogLike(data, mixGaussEst)
        print('Log Likelihood After Iter {} : {:4.3f}\n'.format(cIter, logLike))

    return mixGaussEst